In [1]:
from keras.layers import Input, Dense, Permute, multiply, GlobalAveragePooling1D, Reshape, concatenate
import tensorflow.keras.backend as K
import numpy as np
from keras import optimizers, Model
from keras.utils import Sequence
from numpy.random import seed
import math
from keras.callbacks import EarlyStopping,ModelCheckpoint


def squeeze_excite_block1(tensor, ratio=16):
    init = tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init._keras_shape[channel_axis]
    se_shape = (1,filters)
    init1 = Reshape(target_shape=(se_shape))(init)    
    se = GlobalAveragePooling1D()(init1)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    
    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se) 
    se_reshape = (filters,)
    se = Reshape(target_shape=(se_reshape))(se)
    x = multiply([init, se])
    return x


Using TensorFlow backend.
C:\Users\owner\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\owner\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\owner\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\owner\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [7]:
import h5py
# Reading Patches.
f = h5py.File('Syn_Example.mat', 'r')
dataNoise= f.get('X')
dataNoise = np.array(dataNoise)
INPUT_SIZE1 = dataNoise.shape[0]
INPUT_SIZE2 = dataNoise.shape[1]
input_img = Input(shape=(INPUT_SIZE2,))

# Building the DIP model.
D1 = 128
D2 = int(D1/2)
D3 = int(D2/2)
D4 = int(D3/2)
D5 = int(D4/2)
D6 = int(D5/2)
D7 = int(D6/2)



# Encoder
encoded1 = Dense(D1,  activation='elu',name='e1')(input_img)
encoded1x = squeeze_excite_block1(encoded1,D1)

encoded2 = Dense(D2,  activation='elu',name='e2')(encoded1)
encoded2x = squeeze_excite_block1(encoded2,D2)

encoded3 = Dense(D3,  activation='elu',name='e3')(encoded2)
encoded3x = squeeze_excite_block1(encoded3,D3)

encoded4 = Dense(D4,  activation='elu',name='e4')(encoded3)
encoded4x = squeeze_excite_block1(encoded4,D4)

encoded5 = Dense(D5,  activation='elu',name='e5')(encoded4)
encoded5x = squeeze_excite_block1(encoded5,D5)

encoded6 = Dense(D6,  activation='elu',name='e6')(encoded5)
encoded6x = squeeze_excite_block1(encoded6,D6)


# Decoder
decoded1aa = Dense(D6,  activation='elu',name='d1')(encoded6)
decoded1aa = squeeze_excite_block1(decoded1aa,D6)
decoded1aa = concatenate([decoded1aa,encoded6x])

decoded1a = Dense(D5,  activation='elu',name='d2')(decoded1aa)
decoded1a = squeeze_excite_block1(decoded1a,D5)
decoded1a = concatenate([decoded1a,encoded5x])

decoded1 = Dense(D4,  activation='elu',name='d3')(decoded1a)
decoded1 = squeeze_excite_block1(decoded1,D4)
decoded1 = concatenate([decoded1,encoded4x])

decoded2 = Dense(D3,  activation='elu',name='d4')(decoded1)
decoded2 = squeeze_excite_block1(decoded2,D3)
decoded2 = concatenate([decoded2,encoded3x])

decoded3 = Dense(D2,  activation='elu',name='d5')(decoded2)
decoded3 = squeeze_excite_block1(decoded3,D2)
decoded3 = concatenate([decoded3,encoded2x])

decoded4 = Dense(D1,  activation='elu',name='d6')(decoded3)
decoded4 = squeeze_excite_block1(decoded4,D1)
decoded4 = concatenate([decoded4,encoded1x])

# Final Layer
decoded = Dense(INPUT_SIZE2, activation='linear')(decoded4)





    

            
autoencoder = Model(input_img, decoded)
sgd = optimizers.adam(lr=0.0001)
autoencoder.compile(optimizer=sgd, loss='mse')

autoencoder.summary()

# Addig stopping condition
es=EarlyStopping(monitor='loss',mode='min',verbose=1,patience=5)
mc=ModelCheckpoint('best_model.h5',monitor='loss',mode='min',save_best_only=True)


batch = 128
history = autoencoder.fit(dataNoise,dataNoise, epochs=50, batch_size=batch,callbacks=[es,mc],shuffle=False)




Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 3375)         0                                            
__________________________________________________________________________________________________
e1 (Dense)                      (None, 128)          432128      input_4[0][0]                    
__________________________________________________________________________________________________
e2 (Dense)                      (None, 64)           8256        e1[0][0]                         
__________________________________________________________________________________________________
e3 (Dense)                      (None, 32)           2080        e2[0][0]                         
____________________________________________________________________________________________

Epoch 1/50
36288/36288 [==============================] - 12s 330us/step - loss: 0.0592
Epoch 2/50
36288/36288 [==============================] - 10s 268us/step - loss: 0.0499
Epoch 3/50
36288/36288 [==============================] - 9s 251us/step - loss: 0.0467
Epoch 4/50
36288/36288 [==============================] - 10s 264us/step - loss: 0.0449
Epoch 5/50
36288/36288 [==============================] - 10s 267us/step - loss: 0.0438
Epoch 6/50
36288/36288 [==============================] - 9s 258us/step - loss: 0.0430
Epoch 7/50
36288/36288 [==============================] - 9s 261us/step - loss: 0.0424
Epoch 8/50
36288/36288 [==============================] - 9s 258us/step - loss: 0.0419
Epoch 9/50
36288/36288 [==============================] - 9s 258us/step - loss: 0.0416
Epoch 10/50
36288/36288 [==============================] - 9s 258us/step - loss: 0.0413
Epoch 11/50
36288/36288 [==============================] - 9s 259us/step - loss: 0.0410
Epoch 12/50
36288/36288 [============

In [8]:
# Predict
from keras.models import load_model
import h5py
import scipy.io

model = load_model('best_model.h5')
out = model.predict(dataNoise, batch_size=batch)
scipy.io.savemat('Syn_Example_outpatch.mat', mdict={'out': out})


